In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [2]:
list=[['On',1],['Off',2],['On',3],['On',4],['On',5],['Off',6],['Off',7],['On',8],['On',9],['On',10],['Off',11],['Off',12],['On',13],['On',14],['Off',15]]
df= spark.createDataFrame(list,['Signal_Value','Timestamp'])
df.show()

+------------+---------+
Signal_Value|Timestamp|
+------------+---------+
 On| 1|
 Off| 2|
 On| 3|
 On| 4|
 On| 5|
 Off| 6|
 Off| 7|
 On| 8|
 On| 9|
 On| 10|
 Off| 11|
 Off| 12|
 On| 13|
 On| 14|
 Off| 15|
+------------+---------+

In [3]:
express="""filter(count, x-> x>1)"""
w=Window().partitionBy("Signal_Value").orderBy("Timestamp")
w2=Window().partitionBy("Signal_Value").orderBy("Timestamp").rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)
df1=df.withColumn("rank", F.rank().over(w)).withColumn("rank1", F.col("Timestamp")- F.lag("Timestamp").over(w))\
.withColumn("count", F.collect_list("rank1").over(w)).withColumn("max", F.last("rank").over(w2)).filter(F.col("rank")==F.col("max")).withColumn("groupcount", F.size(F.expr(express))+1)\
.drop("rank","rank1","count","max","timestamp")

In [4]:
df1.show()

+------------+----------+
Signal_Value|groupcount|
+------------+----------+
 On| 4|
 Off| 4|
+------------+----------+

In [5]:
l=df1.collect()

In [6]:
d=[]
for i,j in l:
  d.append([i,j])
  

In [7]:
d

Out[214]: [['On', 3], ['Off', 4]]

In [8]:
train,test=df.randomSplit([0.6,0.4],seed=24)

In [9]:
train.show()
test.show()

+------------+---------+
Signal_Value|Timestamp|
+------------+---------+
 Off| 2|
 On| 3|
 On| 8|
 On| 9|
 On| 10|
 Off| 12|
 On| 13|
 Off| 15|
 On| 14|
+------------+---------+

+------------+---------+
Signal_Value|Timestamp|
+------------+---------+
 Off| 1|
 On| 4|
 On| 5|
 Off| 6|
 Off| 7|
 Off| 11|
+------------+---------+

In [10]:

spark.conf.set("spark.sql.shuffle.partitions", "300")

In [11]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Word") \
.getOrCreate()


#spark.conf.set("spark.sql.shuffle.partitions", "120")

In [12]:
spark.conf.get("spark.sql.default.parallelism")

Out[313]: '10'

In [13]:
spark.conf.set("spark.sql.shuffle.partitions", "120")

In [14]:
spark.conf.set("spark.sql.default.parallelism", "10")

In [15]:
spark.conf.get("spark.sql.default.parallelism")

Out[312]: '10'

In [16]:
df.rdd.getNumPartitions()

Out[223]: 8

In [17]:
df2=df1.coalesce(1)

In [18]:
df2.rdd.getNumPartitions()

Out[242]: 1

In [19]:
def f(person):
    print(person.groupcount, end='')
df2.foreach(f)

In [20]:
for w in df2.toLocalIterator():
    print(w)

Row(Signal_Value='On', groupcount=3)
Row(Signal_Value='Off', groupcount=4)

In [21]:
for i,j in df1.take(2):
  print([i,j])

['On', 3]
['Off', 4]

In [23]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

df = spark.createDataFrame(
    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)], ("id", "v"))
@pandas_udf("id long, v double", PandasUDFType.GROUPED_MAP)
def mean_value(pdf):
    v = pdf.v
    id =pdf.id
    return pdf.assign(v = (v.mean())-id)

df1=df.groupby("id").apply(mean_value)



In [24]:
def plus(df):
  print(df.v)
df1.foreach(plus)

In [25]:
for i in df1.toLocalIterator():
  print(i)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-693087847104222> in <module> 
 ----> 1 for i in df1 . toLocalIterator ( ) [ 0 ] : 
 2 print ( i ) 

 TypeError : 'itertools.chain' object is not subscriptable

In [26]:
df1.show()

+---+---+
 id| v|
+---+---+
 1|0.5|
 1|0.5|
 2|4.0|
 2|4.0|
 2|4.0|
+---+---+

In [27]:
spark.conf.get("spark.sql.default.parallelism")

Out[303]: '10'

In [28]:
spark.conf.get("spark.sql.shuffle.partitions")

Out[311]: '150'

In [29]:
from pyspark.sql.types import *
from pyspark.sql import functions as F
list=[['QUALITY ITEM'],
      ['NOT'],
      ['SANDBOX ITEM'],
      ['NOT'],
      ['PRODUCTIONITEM'],
     ['NOT'],
     ['ITEMTEST']]

applymapping_DyF= spark.createDataFrame(list, ['status_purpose'])

applymapping_DyF.withColumn('check_status_ind', F.when(((F.col('status_purpose').cast(StringType()).contains("ITEM"))), "Y").otherwise("N")).show()

+--------------+----------------+
status_purpose|check_status_ind|
+--------------+----------------+
 QUALITY ITEM| Y|
 NOT| N|
 SANDBOX ITEM| Y|
 NOT| N|
PRODUCTIONITEM| Y|
 NOT| N|
 ITEMTEST| Y|
+--------------+----------------+

In [30]:
%scala
applymapping_DyF.withColumn($'check_status_ind', F.when(((F.col('status_purpose').cast(StringType()).contains("ITEM"))), "Y").otherwise("N")).show()

In [31]:
arrayData = [
        ('James',[{'hair':'black','eye':'brown','lastname':'Hashmi'}]),
        ('Michael',[{'hair':'brown','eye':'blue','lastname':'Turary'}]),
        ('Robert',[{'hair':'red','eye':'brown','lastname':'Balthakrey'}]),
        ('Washington',[{'hair':'red','eye':'brown','lastname':'Balthakrey'}]),
        ('Jefferson',[{'hair':'red','eye':'brown','lastname':'Balthakrey'}])]

df = spark.createDataFrame(data=arrayData, schema = ['name','properties'])
df.printSchema()
df.show()

root
-- name: string (nullable = true)
-- properties: array (nullable = true)
 |-- element: map (containsNull = true)
 | |-- key: string
 | |-- value: string (valueContainsNull = true)

+----------+--------------------+
 name| properties|
+----------+--------------------+
 James|[[eye -> brown, h...|
 Michael|[[eye -> blue, ha...|
 Robert|[[eye -> brown, h...|
Washington|[[eye -> brown, h...|
 Jefferson|[[eye -> brown, h...|
+----------+--------------------+

In [32]:
df.select("name",F.explode("properties").alias("e")).show(truncate=False)
#df1.selectExpr("e.lastname","e.hair","e.eye").show()

+----------+---------------------------------------------------+
name |e |
+----------+---------------------------------------------------+
James |[eye -> brown, hair -> black, lastname -> Hashmi] |
Michael |[eye -> blue, hair -> brown, lastname -> Turary] |
Robert |[eye -> brown, hair -> red, lastname -> Balthakrey]|
Washington|[eye -> brown, hair -> red, lastname -> Balthakrey]|
Jefferson |[eye -> brown, hair -> red, lastname -> Balthakrey]|
+----------+---------------------------------------------------+

In [33]:
list=[[1,'event1',10,'2016-05-01'],
      [2,'event2',100,'2016-05-01'],
      [1,'event3',20,'2016-05-02'],
      [1,'event1',15,'2016-05-03'],
      [3,'event1',13,'2016-05-04'],
      [1,'event2',12,'2016-05-04'],
      [1,'event3',11,'2016-05-05']]
df2= spark.createDataFrame(list,['col1','col2','col3','Date'])
df2.show()

list=[['1event110',12],
      ['1event115',23],
      ['1event212',45],
      ['yoyoyoyooyoyoyoyo',454],
      ['yoyooyoyoyooyoyoy',454],
      ['yooyoyooyoyooyoy',454],
      ['yoooyoyoyoyoyoyo',878]]
df1= spark.createDataFrame(list,['col5','sale'])
df1.show()

+----+------+----+----------+
col1| col2|col3| Date|
+----+------+----+----------+
 1|event1| 10|2016-05-01|
 2|event2| 100|2016-05-01|
 1|event3| 20|2016-05-02|
 1|event1| 15|2016-05-03|
 3|event1| 13|2016-05-04|
 1|event2| 12|2016-05-04|
 1|event3| 11|2016-05-05|
+----+------+----+----------+

+-----------------+----+
 col5|sale|
+-----------------+----+
 1event110| 12|
 1event115| 23|
 1event212| 45|
yoyoyoyooyoyoyoyo| 454|
yoyooyoyoyooyoyoy| 454|
 yooyoyooyoyooyoy| 454|
 yoooyoyoyoyoyoyo| 878|
+-----------------+----+

In [34]:
arr=['col1','col2','col3']
df3 = df2.join(df1, F.concat(*(F.col(col) for col in arr)) == df1.col5 )

In [35]:
df3.show()

+----+------+----+----------+---------+----+
col1| col2|col3| Date| col5|sale|
+----+------+----+----------+---------+----+
 1|event2| 12|2016-05-04|1event212| 45|
 1|event1| 15|2016-05-03|1event115| 23|
 1|event1| 10|2016-05-01|1event110| 12|
+----+------+----+----------+---------+----+

In [36]:
df2.withColumn("col5", F.concat(*((F.col(col).cast("string")) for col in arr))).show()

+----+------+----+----------+----------+
col1| col2|col3| Date| col5|
+----+------+----+----------+----------+
 1|event1| 10|2016-05-01| 1event110|
 2|event2| 100|2016-05-01|2event2100|
 1|event3| 20|2016-05-02| 1event320|
 1|event1| 15|2016-05-03| 1event115|
 3|event1| 13|2016-05-04| 3event113|
 1|event2| 12|2016-05-04| 1event212|
 1|event3| 11|2016-05-05| 1event311|
+----+------+----+----------+----------+

In [37]:
df1.dtypes

Out[68]: [('col5', 'string'), ('sale', 'bigint')]